In [4]:
# Reinitialize the connection to set up tables with sample data
conn = sqlite3.connect('example_database.db')
cursor = conn.cursor()

# Drop tables if they exist to start fresh
cursor.execute("DROP TABLE IF EXISTS SALES")
cursor.execute("DROP TABLE IF EXISTS ITEMS")
cursor.execute("DROP TABLE IF EXISTS CUSTOMERS")

# Create SALES table
cursor.execute('''
CREATE TABLE SALES (
    Date TEXT,
    Order_id INTEGER,
    Item_id INTEGER,
    Customer_id INTEGER,
    Quantity INTEGER,
    Revenue REAL
)
''')

# Create ITEMS table
cursor.execute('''
CREATE TABLE ITEMS (
    Item_id INTEGER PRIMARY KEY,
    Item_name TEXT,
    Price REAL,
    Department TEXT
)
''')

# Create CUSTOMERS table
cursor.execute('''
CREATE TABLE CUSTOMERS (
    Customer_id INTEGER PRIMARY KEY,
    First_name TEXT,
    Last_name TEXT,
    Address TEXT
)
''')

# Insert sample data into SALES table
sales_data = [
    ('2023-03-18', 1, 101, 1, 2, 20.00),
    ('2023-03-18', 2, 102, 2, 1, 15.00),
    ('2023-01-10', 3, 103, 1, 3, 30.00),
    ('2023-01-15', 4, 101, 3, 1, 10.00),
    ('2022-06-05', 5, 101, 2, 1, 10.00),
    ('2022-06-06', 6, 102, 1, 1, 15.00)
]
cursor.executemany('INSERT INTO SALES VALUES (?, ?, ?, ?, ?, ?)', sales_data)

# Insert sample data into ITEMS table
items_data = [
    (101, 'ItemA', 10.00, 'Department1'),
    (102, 'ItemB', 15.00, 'Department2'),
    (103, 'ItemC', 12.50, 'Department1')
]
cursor.executemany('INSERT INTO ITEMS VALUES (?, ?, ?, ?)', items_data)

# Insert sample data into CUSTOMERS table
customers_data = [
    (1, 'John', 'Doe', '123 Elm St'),
    (2, 'Jane', 'Smith', '456 Oak St'),
    (3, 'Alice', 'Johnson', '789 Maple St')
]
cursor.executemany('INSERT INTO CUSTOMERS VALUES (?, ?, ?, ?)', customers_data)

# Commit changes
conn.commit()


# 1. Total number of orders completed on 18th March 2023
query1 = """
SELECT COUNT(DISTINCT Order_id) AS total_orders
FROM SALES
WHERE Date = '2023-03-18';
"""
cursor.execute(query1)
total_orders_18_march = cursor.fetchone()[0]

# 2. Total number of orders completed on 18th March 2023 with the first name ‘John’ and last name ‘Doe’
query2 = """
SELECT COUNT(DISTINCT s.Order_id) AS total_orders
FROM SALES s
JOIN CUSTOMERS c ON s.Customer_id = c.Customer_id
WHERE s.Date = '2023-03-18'
  AND c.First_name = 'John'
  AND c.Last_name = 'Doe';
"""
cursor.execute(query2)
total_orders_john_doe_18_march = cursor.fetchone()[0]

# 3. Total number of customers that purchased in January 2023 and the average amount spent per customer
query3 = """
SELECT COUNT(DISTINCT Customer_id) AS total_customers,
       AVG(total_spent) AS avg_amount_spent
FROM (
    SELECT Customer_id, SUM(Revenue) AS total_spent
    FROM SALES
    WHERE Date BETWEEN '2023-01-01' AND '2023-01-31'
    GROUP BY Customer_id
) AS customer_spending;
"""
cursor.execute(query3)
total_customers_jan, avg_amount_spent_jan = cursor.fetchone()

# 4. Departments that generated less than $600 in 2022
query4 = """
SELECT i.Department, SUM(s.Revenue) AS total_revenue
FROM SALES s
JOIN ITEMS i ON s.Item_id = i.Item_id
WHERE strftime('%Y', s.Date) = '2022'
GROUP BY i.Department
HAVING SUM(s.Revenue) < 600;
"""
cursor.execute(query4)
departments_less_than_600_2022 = cursor.fetchall()

# 5. Most and least revenue generated by an order
query5 = """
SELECT MAX(order_revenue) AS max_revenue, MIN(order_revenue) AS min_revenue
FROM (
    SELECT Order_id, SUM(Revenue) AS order_revenue
    FROM SALES
    GROUP BY Order_id
) AS order_totals;
"""
cursor.execute(query5)
max_revenue, min_revenue = cursor.fetchone()

# 6. Orders that were purchased in the most lucrative order
query6 = """
WITH lucrative_order AS (
    SELECT Order_id, SUM(Revenue) AS total_revenue
    FROM SALES
    GROUP BY Order_id
    ORDER BY total_revenue DESC
    LIMIT 1
)
SELECT s.*
FROM SALES s
JOIN lucrative_order lo ON s.Order_id = lo.Order_id;
"""
cursor.execute(query6)
most_lucrative_order = cursor.fetchall()

conn.close()

# Display
(total_orders_18_march, total_orders_john_doe_18_march,
 total_customers_jan, avg_amount_spent_jan,
 departments_less_than_600_2022, max_revenue,
 min_revenue, most_lucrative_order)

(2,
 1,
 2,
 20.0,
 [('Department1', 10.0), ('Department2', 15.0)],
 30.0,
 10.0,
 [('2023-01-10', 3, 103, 1, 3, 30.0)])